<a href="https://colab.research.google.com/github/Rob192/verbatim/blob/master/trial_lda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [12]:
df = pd.read_csv('https://raw.githubusercontent.com/Rob192/verbatim/master/COVID-19-RETEX_5-25-2020_pavel.csv', sep = ',')
df.head()

,N°Obs,1. Quel est votre ministère d'appartenance ?,2. Quelle est votre fonction ?,3. Précisez votre fonction,"4. Quel est le nom de votre direction, sous-direction ou service",5. Quelle est votre région (pour les administrations territorial,6. A1. Considérez-vous avoir eu rapidement les moyens de mobilis,7. A1a. Pour quelles raisons ?,8. A2. Dans quel délai avez-vous eu connaissance des moyens numé,9. A3. Comment notez-vous l'efficacité de vos moyens de télétrav,10. A4. Quelles initiatives complémentaires avez-vous prises pou,11. Messagerie du ministère,12. Messagerie autre (ex: gmail),13. Messagerie instantanée interministérielle (Tchap),14. Messagerie instantanée autre (ex: WhatsApp),15. Visio-conférence du ministère,16. Viso-conférence interministériel (WebConf),17. Visio-conférence autre (ex: Skype),18. Outils collaboratifs du ministère,"19. Outils collaboratifs interministériels (OSMOSE, PLANO, RESAN",20. Outils collaboratifs autres,21. Audio-conférence téléphonique,22. B2. Avez-vous eu recours à des ressources ou compétences add,"23. B2a. Si oui, lesquelles et comment les avez-vous activées ?",24. B3. A quelles principales difficultés avez-vous fait face po,25. C1. Considérez-vous que votre direction disposait d'une cult,26. C2. Quels moyens de communication internes avez-vous mis en,"27. C2a. Si ""autre"", lesquelles ?","28. C3. Avez-vous détecté des agents mis en difficulté, y compri",29. C3a. Que pouvez-vous en dire ?,30. C4. Entendez-vous conserver au-delà de la crise des pratique,"31. C4a. Si oui, lesquelles ?",32. C5. Que souhaitez-vous ajouter sur les impacts du numérique,33. Clé,34. Date de saisie,35. Date de dernière modification,36. Date de dernier enregistrement,37. Temps de saisie,38. Langue,39. Progression,40. Origine,41. Appareil utilisé pour la saisie
0,1,Services du Premier ministre,Autre,secrétaire générale,NaN,NaN,Oui,NaN,Dès la 1ère semaine,4,NaN,Très utile,Pas utile,Non utilisé,Non utilisé,Non utilisé,Non utilisé,Utile,Non utilisé,Non utilisé,Utile,Très utile,Non,NaN,insuffisance de la bande passante; lenteurs;,Oui mais insuffisamment,Courriels / Lettre d'information;Enquête en li...,NaN,Oui,handicap; isolement;,Oui,fluidité des validations électroniques; visio ...,NaN,66DP-LXBD,28/04/2020 15:01:25,NaN,28/04/2020 15:05:50,264.940515,FR,Terminé,Direct,PC
1,2,Services du Premier ministre,Directeur.trice (centrale),NaN,CIVEN,NaN,Pas suffisamment,Manque de postes PC portables avec accès télét...,Dès la 1ère semaine,5,Ensemble du travail du CIVEN se fait par télét...,Très utile,Utile,Non utilisé,Utile,Non utilisé,Non utilisé,Utile,Non utilisé,Non utilisé,Utile,Très utile,Oui,Travail sur l'application informatique du CIVE...,manque de postes informatiques télétravail pou...,Oui mais insuffisamment,Courriels / Lettre d'information;Réseau social...,NaN,Non,NaN,Oui,Recours au télétravail et mise place d'audioco...,"La période de confinement, malgré le soutien d...",TNTW-XFV4,28/04/2020 15:06:54,NaN,28/04/2020 15:18:24,690.338652,FR,Terminé,Direct,PC
2,3,Services du Premier ministre,Autre,Secrétaire générale,Délégation interministérielle aux jeux Olympiq...,NaN,Oui,NaN,Dès la 1ère semaine,5,Maintien de réunions de service hebdomadaires ...,Très utile,Pas utile,Inconnu,Très utile,Non utilisé,Non utilisé,Très utile,Non utilisé,Inconnu,Non utilisé,Très utile,Non,NaN,Aucune difficulté,Oui,Courriels / Lettre d'information;Réseau social...,NaN,Non,NaN,Je ne sais pas encore,NaN,Néant,AK5M-Y4WQ,28/04/2020 15:11:49,NaN,28/04/2020 15:24:11,742.120496,FR,Terminé,Direct,PC
3,4,Services du Premier ministre,Autre,Secrétaire général,Ecole nationale d'administration,NaN,Oui,NaN,Jamais,5,"En tant qu'établissement public administratif,...",Non utilisé,Non utilisé,Non utilisé,Non utilisé,Non utilisé,Non utilisé,Très utile,Non utilisé,Non utilisé,Très utile,Très utile,Non,NaN,"Les outils ne sont pas en cause, comme indiqué...",Oui,Courriels / Lettre d'information;Site Web;Web ...,Le dialogue social. J'ai réuni les organisatio...,

In [0]:
cols = df.columns

In [14]:
cols

Index(['N°Obs', '1. Quel est votre ministère d'appartenance ?',
       '2. Quelle est votre fonction ?', '3. Précisez votre fonction',
       '4. Quel est le nom de votre direction, sous-direction ou service',
       '5. Quelle est votre région (pour les administrations territorial',
       '6. A1. Considérez-vous avoir eu rapidement les moyens de mobilis',
       '7. A1a. Pour quelles raisons ?',
       '8. A2. Dans quel délai avez-vous eu connaissance des moyens numé',
       '9. A3. Comment notez-vous l'efficacité de vos moyens de télétrav',
       '10. A4. Quelles initiatives complémentaires avez-vous prises pou',
       '11. Messagerie du ministère', '12. Messagerie autre (ex: gmail)',
       '13. Messagerie instantanée interministérielle (Tchap)',
       '14. Messagerie instantanée autre (ex: WhatsApp)',
       '15. Visio-conférence du ministère',
       '16. Viso-conférence interministériel (WebConf)',
       '17. Visio-conférence autre (ex: Skype)',
       '18. Outils collabora

In [0]:
df = df['31. C4a. Si oui, lesquelles ?']
df.columns = ["bonnes_pratiques"]

In [18]:
df.head()

0    fluidité des validations électroniques; visio ...
1    Recours au télétravail et mise place d'audioco...
2                                                  NaN
3    Il est prévu dans le PRAP une période de maint...
4                                                  NaN
Name: 31. C4a. Si oui, lesquelles ?, dtype: object

In [0]:
df.dropna(inplace=True)

In [21]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lemmatizer import Lemmatizer
#from spacy.lang.en.stop_words import STOP_WORDS

from tqdm import tqdm_notebook as tqdm
from pprint import pprint

     |████████████████████████████████| 88.6MB 1.1MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.2.5-cp36-none-any.whl size=90338488 sha256=495bd0f166daa67576155b2c8b6b60ad968e2e7108b36df8359bd49f52324058
  Stored in directory: /tmp/pip-ephem-wheel-cache-pvgp078r/wheels/c6/18/b6/f628642acc7872a53cf81269dd1c394d96da69564ccfac5425
Successfully built fr-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


In [24]:
import spacy.cli
spacy.cli.download("fr_core_news_md")

✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


In [0]:
nlp = spacy.load('fr_core_news_md')

In [0]:
# My list of stop words.
stop_list = ["\n "]

# Updates spaCy's default stop words list with my additional words. 
nlp.Defaults.stop_words.update(stop_list)

# Iterates over the words in the stop words list and resets the "is_stop" flag.
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True

In [66]:
txt = df[1]
doc = nlp(txt)
txt

"Recours au télétravail et mise place d'audioconférence pour certaines réunions."

In [67]:
doc

Recours au télétravail et mise place d'audioconférence pour certaines réunions.

In [68]:
def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)
lemmatizer(doc)

recours au télétravail et mise place de audioconférence pour certain réunion .

In [69]:
def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc
remove_stopwords(doc)

['Recours', 'télétravail', 'mise', 'place', 'audioconférence', 'réunions']

In [0]:
nlp.add_pipe(lemmatizer,name='lemmatizer',after='ner')
nlp.add_pipe(remove_stopwords, name="stopwords", last=True)

In [71]:
df.values

array(['fluidité des validations électroniques; visio conférence',
       "Recours au télétravail et mise place d'audioconférence pour certaines réunions.",
       "Il est prévu dans le PRAP une période de maintien du télétravail avec un objectif cible global de 80% (soit quatre jours hebdomadaires de télétravail pour une quotité à temps plein). Avant la crise, l'objectif était d'atteindre un jour de télétravail hebdomadaire, pour certains agents, cette périodicité était de deux jours, pour une période limitée et reconductible.\nLe nouvel objectif apparait dès à présent de deux jours hebdomadaires, avec une possibilité de trois jours, voire quatre jours à titre exceptionnel pour une durée limitée. Ces éléments ne sont pas encore formalisés, mais prennent tournure.",
       'Toutes sauf la validation dématérialisée',
       'Il faut probablement envisager un mode degrade ou semi permanent d\'administration "hors les murs" car : 1. des crises comme cella du COVID19 vont sans doute se rep

In [72]:
doc_list = []
# Iterates through each article in the corpus.
for txt in tqdm(df.values):
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(txt)
    doc_list.append(pr)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [73]:
doc_list

[['fluidité', 'validation', 'électronique', 'visio', 'conférenc'],
 ['recours', 'télétravail', 'mise', 'place', 'audioconférence', 'réunion'],
 ['prévoir',
  'prap',
  'période',
  'maintien',
  'télétravail',
  'objectif',
  'cibl',
  'global',
  '80',
  'pourcent',
  'jour',
  'hebdomadaire',
  'télétravail',
  'quotité',
  'temps',
  'crise',
  'objectif',
  'atteindre',
  'jour',
  'télétravail',
  'hebdomadaire',
  'agent',
  'périodicité',
  'jour',
  'période',
  'limité',
  'reconductible',
  'objectif',
  'apparer',
  'présent',
  'jour',
  'hebdomadaire',
  'possibilité',
  'jour',
  'voire',
  'jour',
  'titre',
  'exceptionnel',
  'durée',
  'limité',
  'élément',
  'formaliser',
  'prendre',
  'tournur'],
 ['validation', 'dématérialiser'],
 ['falloir',
  'probablement',
  'envisager',
  'mode',
  'degrad',
  'semi',
  'permanent',
  'administration',
  'mur',
  '1',
  'crise',
  'cella',
  'COVID19',
  'aller',
  'doute',
  'reproduire',
  'PCA',
  'résilience',
  '2',
  '

In [0]:
# Creates, which is a mapping of word IDs to words.
words = corpora.Dictionary(doc_list)

# Turns each document into a bag of words.
corpus = [words.doc2bow(doc) for doc in doc_list]

In [0]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=3, 
                                           random_state=2,
                                           update_every=1,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [76]:
pprint(lda_model.print_topics(num_words=10))

[(0,
  '0.042*"télétravail" + 0.023*"réunion" + 0.020*"travail" + 0.018*"distance" '
  '+ 0.016*"visio" + 0.015*"audio" + 0.015*"outil" + 0.014*"  " + '
  '0.011*"déplacement" + 0.011*"recours"'),
 (1,
  '0.039*"télétravail" + 0.034*"visio" + 0.022*"conférence" + 0.017*"agent" + '
  '0.015*"travail" + 0.014*"audio" + 0.014*"réunion" + 0.013*"outil" + '
  '0.007*"distance" + 0.007*"dématérialisation"'),
 (2,
  '0.081*"télétravail" + 0.020*"agent" + 0.017*"jour" + '
  '0.011*"visioconférence" + 0.010*"semaine" + 0.008*"place" + 0.008*"pouvoir" '
  '+ 0.007*"permettre" + 0.007*"crise" + 0.006*"développer"')]
